In [2]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import matplotlib.font_manager as fm
import IPython.display as disp
import squarify

# Business

파일 불러오기

In [2]:
file_path_business = "../yelp_dataset/yelp_academic_dataset_business.json"

# total lines 미리 세기 (tqdm 활용)
with open(file_path_business, 'r', encoding='utf-8') as f:
    total_lines = sum(1 for _ in f)

data = []
with open(file_path_business, 'r', encoding='utf-8') as f:
    for line in tqdm(f, total=total_lines, desc="Loading businesses"):
        data.append(json.loads(line))

df_business = pd.DataFrame(data)

Loading businesses: 100%|██████████| 150346/150346 [00:01<00:00, 106377.51it/s]


필요 없는 칼럼 제거

In [3]:
drop_cols_business = ['name','address','postal_code','latitude','longitude','is_open','attributes','hours']
df_business = df_business.drop(columns=drop_cols_business)

음식관련 업종만 필터링

In [4]:
def load_categories(fp):
    with open(fp,'r',encoding='utf-8') as f:
        return set(line.strip().lower() for line in f if line.strip())
food_categories = load_categories('../src/food.txt')
restaurant_categories = load_categories('../src/restaurant.txt')
target_categories = food_categories.union(restaurant_categories)

def category_match(row):
    if isinstance(row, str) and row.strip():  # 빈 문자열도 고려하여 추가
        biz_categories = set(cat.strip().lower() for cat in row.split(','))
        return bool(biz_categories & target_categories)
    return False

df_business = df_business[df_business['categories'].apply(category_match)]

In [5]:
df_business.rename(columns={'stars': 'business_stars', 'review_count': 'business_review_count'}, inplace=True)

## DataFrame Information

- **Index**: 63,199 entries (from 3 to 150340)
- **Columns**: 6

| #   | Column        | Non-Null Count | Dtype  |
|-----|---------------|----------------|--------|
| 0   | business_id   | 63,199         | object |
| 1   | city          | 63,199         | object |
| 2   | state         | 63,199         | object |
| 3   | business_stars         | 63,199         | float64|
| 4   | business_review_count  | 63,199         | int64  |
| 5   | categories    | 63,199         | object |

**Memory Usage**: 3.4+ MB

# User

In [6]:
file_path_user = "../yelp_dataset/yelp_academic_dataset_user.json"

# total lines 미리 세기 (tqdm 활용)
with open(file_path_user, 'r', encoding='utf-8') as f:
    total_lines = sum(1 for _ in f)

data = []
with open(file_path_user, 'r', encoding='utf-8') as f:
    for line in tqdm(f, total=total_lines, desc="Loading users"):
        data.append(json.loads(line))

df_user = pd.DataFrame(data)

Loading users: 100%|██████████| 1987897/1987897 [00:15<00:00, 131217.81it/s]


In [7]:
drop_cols_user = ['name', 'yelping_since', 'funny', 'cool', 'elite', 'friends', 'fans', 'compliment_hot', 'compliment_more', 'compliment_profile', 'compliment_cute', 'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_cool', 'compliment_funny', 'compliment_writer', 'compliment_photos']
df_user = df_user.drop(columns=drop_cols_user)

In [8]:
df_user.rename(columns={'average_stars': 'user_average_stars', 'useful': 'user_useful', 'review_count': 'user_review_count'}, inplace=True)


## DataFrame Information

- **Index**: 1,987,897 entries (from 0 to 1,987,896)
- **Columns**: 4

| #   | Column             | Dtype   |
|-----|--------------------|---------|
| 0   | user_id            | object  |
| 1   | user_review_count  | int64   |
| 2   | user_useful        | int64   |
| 3   | user_average_stars | float64 |

**Memory Usage**: 60.7+ MB

# review

In [9]:
file_path_review = "../yelp_dataset/yelp_academic_dataset_review.json"

# total lines 미리 세기 (tqdm 활용)
with open(file_path_review, 'r', encoding='utf-8') as f:
    total_lines = sum(1 for _ in f)

data = []
with open(file_path_review, 'r', encoding='utf-8') as f:
    for line in tqdm(f, total=total_lines, desc="Loading reviews"):
        data.append(json.loads(line))

df_review = pd.DataFrame(data)

Loading reviews: 100%|██████████| 6990280/6990280 [00:25<00:00, 274091.16it/s]


In [10]:
drop_cols_review = ['funny', 'cool']
df_review = df_review.drop(columns=drop_cols_review)

In [11]:
df_review.rename(columns={'stars': 'review_stars', 'useful': 'review_useful'}, inplace=True)

## DataFrame Information

- **Index**: 6,990,280 entries (from 0 to 6,990,279)
- **Columns**: 7

| #   | Column         | Dtype   |
|-----|----------------|---------|
| 0   | review_id      | object  |
| 1   | user_id        | object  |
| 2   | business_id    | object  |
| 3   | review_stars   | float64 |
| 4   | review_useful  | int64   |
| 5   | text           | object  |
| 6   | date           | object  |

**Memory Usage**: 373.3+ MB

# Combine data

In [12]:
# df_review와 df_user를 user_id 기준으로 합침
df_dataset = pd.merge(df_review, df_user, on='user_id', how='inner')

# df_review와 df_business를 business_id 기준으로 합침
df_dataset = pd.merge(df_dataset, df_business, on='business_id', how='inner')

In [13]:
# 청크 크기 설정
chunk_size = 100000  # 원하는 청크 크기 (예: 10만 행씩)

# 파일 저장
with open('../output/dataset_1.json', 'w', encoding='utf-8') as f:
    for i in range(0, len(df_dataset), chunk_size):
        # 청크 단위로 데이터를 가져오기
        chunk = df_dataset.iloc[i:i+chunk_size]
        # 첫 번째 청크에는 '[' 추가, 이후 청크에는 ','로 이어서 추가
        if i == 0:
            chunk.to_json(f, orient='records', lines=True, force_ascii=False)
        else:
            chunk.to_json(f, orient='records', lines=True, force_ascii=False, date_format='iso')

## DataFrame Information

- **Index**: 5,069,533 entries (from 0 to 5,069,532)
- **Columns**: 15

| #   | Column                    | Dtype   |
|-----|---------------------------|---------|
| 0   | review_id                 | object  |
| 1   | user_id                   | object  |
| 2   | business_id               | object  |
| 3   | review_stars              | float64 |
| 4   | review_useful             | int64   |
| 5   | text                      | object  |
| 6   | date                      | object  |
| 7   | user_review_count         | int64   |
| 8   | user_useful               | int64   |
| 9   | user_average_stars        | float64 |
| 10  | city                      | object  |
| 11  | state                     | object  |
| 12  | business_stars            | float64 |
| 13  | business_review_count     | int64   |
| 14  | categories                | object  |

**Memory Usage**: 580.2+ MB

In [14]:
# df_dataset의 각 컬럼별 결측치 수 확인
missing_data = df_dataset.isnull().sum()
print(missing_data)

review_id                0
user_id                  0
business_id              0
review_stars             0
review_useful            0
text                     0
date                     0
user_review_count        0
user_useful              0
user_average_stars       0
city                     0
state                    0
business_stars           0
business_review_count    0
categories               0
dtype: int64


# Step 2

- 로딩 시간 : 1차 약 4m 50s | 2차 6m

In [15]:
file_path_dataset_1 = "../output/dataset_1.json"

# 데이터를 읽어들여 DataFrame으로 변환
data = []
with open(file_path_dataset_1, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

# DataFrame 생성
df_dataset_1 = pd.DataFrame(data)

필요 없는 칼럼 제거
- 소요 시간 : 1m 40s

In [16]:
drop_cols_dataset_1 = ['user_review_count','user_useful','user_average_stars','city','business_stars','business_review_count','categories']
df_dataset_1 = df_dataset_1.drop(columns=drop_cols_dataset_1)

date 타임스탬프로 변환
- 소요 시간 : 53s

In [17]:
# 'date' 컬럼을 pandas datetime으로 변환
df_dataset_1['date'] = pd.to_datetime(df_dataset_1['date'])

# 밀리초 단위의 타임스탬프로 변환
df_dataset_1['date'] = df_dataset_1['date'].apply(lambda x: int(x.timestamp() * 1000))

별점 형식 float64 -> int64로 변환

In [18]:
# 소수점을 반올림하여 int64로 변환
df_dataset_1['review_stars'] = df_dataset_1['review_stars'].round().astype('int64')

In [19]:
df_dataset_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5069533 entries, 0 to 5069532
Data columns (total 8 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   review_id      object
 1   user_id        object
 2   business_id    object
 3   review_stars   int64 
 4   review_useful  int64 
 5   text           object
 6   date           int64 
 7   state          object
dtypes: int64(3), object(5)
memory usage: 309.4+ MB


In [20]:
# 청크 크기 설정
chunk_size = 100000  # 원하는 청크 크기 (예: 10만 행씩)

# 파일 저장
with open('../output/dataset_2.json', 'w', encoding='utf-8') as f:
    for i in range(0, len(df_dataset_1), chunk_size):
        # 청크 단위로 데이터를 가져오기
        chunk = df_dataset.iloc[i:i+chunk_size]
        # 첫 번째 청크에는 '[' 추가, 이후 청크에는 ','로 이어서 추가
        if i == 0:
            chunk.to_json(f, orient='records', lines=True, force_ascii=False)
        else:
            chunk.to_json(f, orient='records', lines=True, force_ascii=False, date_format='iso')

## DataFrame Information

- **Index**: 5,069,533 entries (from 0 to 5,069,532)
- **Columns**: 8

| #   | Column         | Dtype   |
|-----|----------------|---------|
| 0   | review_id      | object  |
| 1   | user_id        | object  |
| 2   | business_id    | object  |
| 3   | review_stars   | int64   |
| 4   | review_useful  | int64   |
| 5   | text           | object  |
| 6   | date           | int64   |
| 7   | state          | object  |

**Memory Usage**: 309.4+ MB

# Step 3

로딩 시간 : 1차 1m 30s | 2차 3m 59s

In [3]:
file_path_dataset_2 = "../output/dataset_2.json"

# 데이터를 읽어들여 DataFrame으로 변환
data = []
with open(file_path_dataset_2, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

# DataFrame 생성
df_dataset_2 = pd.DataFrame(data)

In [4]:
df_dataset_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5069533 entries, 0 to 5069532
Data columns (total 15 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   review_id              object 
 1   user_id                object 
 2   business_id            object 
 3   review_stars           float64
 4   review_useful          int64  
 5   text                   object 
 6   date                   object 
 7   user_review_count      int64  
 8   user_useful            int64  
 9   user_average_stars     float64
 10  city                   object 
 11  state                  object 
 12  business_stars         float64
 13  business_review_count  int64  
 14  categories             object 
dtypes: float64(3), int64(4), object(8)
memory usage: 580.2+ MB


- 시간 : 1차 55.4s | 2차 3m 10s

In [5]:
# 데이터프레임을 state 기준으로 나누기
df_pa = df_dataset_2[df_dataset_2['state'] == 'PA'].copy()
df_fl = df_dataset_2[df_dataset_2['state'] == 'FL'].copy()
df_la = df_dataset_2[df_dataset_2['state'] == 'LA'].copy()

# 각 df에서 state 컬럼 삭제
df_pa.drop(columns=['state'], inplace=True)
df_fl.drop(columns=['state'], inplace=True)
df_la.drop(columns=['state'], inplace=True)

# 상태 확인용: 초기 데이터 수
initial_pa = len(df_pa)
initial_fl = len(df_fl)
initial_la = len(df_la)

- 시간 : 3m 48s

In [6]:
# 반복 작업: 유저와 비즈니스 리뷰 수가 5개 이상인 것만 남기기
def filter_data(df):
    total_deleted = 0
    while True:
        # 리뷰가 5개 이상인 유저만 남기기
        user_counts = df['user_id'].value_counts()
        filtered_users = user_counts[user_counts >= 5].index
        df_filtered = df[df['user_id'].isin(filtered_users)]

        # 리뷰가 5개 이상인 비즈니스만 남기기
        business_counts = df_filtered['business_id'].value_counts()
        filtered_businesses = business_counts[business_counts >= 5].index
        df_filtered = df_filtered[df_filtered['business_id'].isin(filtered_businesses)]

        # 조건 만족할 때까지 반복
        deleted_rows = len(df) - len(df_filtered)  # 삭제된 데이터 수 계산
        total_deleted += deleted_rows

        if len(df_filtered) == len(df):
            break
        else:
            df = df_filtered

    return df, len(df), total_deleted

# 각 데이터프레임에 필터링 적용
df_pa_filtered, final_pa, deleted_pa = filter_data(df_pa)
df_fl_filtered, final_fl, deleted_fl = filter_data(df_fl)
df_la_filtered, final_la, deleted_la = filter_data(df_la)

In [7]:
# 각 데이터프레임의 결과를 딕셔너리로 정리
data = {
    "State": ["PA", "FL", "LA"],
    "Initial Rows": [initial_pa, initial_fl, initial_la],
    "Remaining Rows": [final_pa, final_fl, final_la],
    "Deleted Rows": [deleted_pa, deleted_fl, deleted_la]
}

# DataFrame 생성
df_summary = pd.DataFrame(data)

# 남은 데이터 비율 계산
df_summary["Remaining Percentage"] = (df_summary["Remaining Rows"] / df_summary["Initial Rows"]) * 100

# 결과 출력
print(df_summary.to_string(index=False))

State  Initial Rows  Remaining Rows  Deleted Rows  Remaining Percentage
   PA       1176504          750446        426058             63.786098
   FL        851457          500892        350565             58.827633
   LA        584475          277766        306709             47.524017


- 3개 지역 데이터 저장
- 시간 : 1차 31.5s | 2차 1m 14s

In [8]:
# df_pa_filtered 저장
df_pa_filtered.to_json('../output/dataset_pa.json', orient='records', lines=True, force_ascii=False)

# df_fl_filtered 저장
df_fl_filtered.to_json('../output/dataset_fl.json', orient='records', lines=True, force_ascii=False)

# df_la_filtered 저장
df_la_filtered.to_json('../output/dataset_la.json', orient='records', lines=True, force_ascii=False)